In [ ]:
import hopsworks
import os
project = hopsworks.login()
mr = project.get_model_registry()

# get model object
model = mr.get_model("xgboost_fraud_online_model", version=1)

In [ ]:
%%writefile /tmp/predict_example.py
import os
import numpy as np
import pandas as pd
import hsfs
import joblib

# On demand feature
from fraud.features.transactions.transactions import transaction_abroad


class Predict(object):

    def __init__(self):
        """ Initializes the serving state, reads a trained model"""        
        # get feature store handle
        fs_conn = hsfs.connection()
        fs = fs_conn.get_feature_store()
        
        # get feature view
        self.fraud_fv = fs.get_feature_view("fraud_model", 1)
        self.profiles_country_fv = fs.get_feature_view("profiles_country", 1)
        
        # initialize serving
        self.fraud_fv.init_serving(1)

        # load the trained model
        self.model = joblib.load(os.environ["ARTIFACT_FILES_PATH"] + "/xgboost_fraud_online_model.pkl")
        
        # initialize monitoring feature group
        self.monitoring_fg = fs.get_feature_group("monitoring_fraud_model", 2)
        
        print("Initialization Complete")

    def predict(self, inputs):
        feature_vector = self.prepare_feature_vector(inputs)
        prediction = self.model.predict(np.asarray(feature_vector).reshape(1, -1)).tolist()
        self.monitor(inputs, feature_vector, prediction[0])
       
        return prediction
    
    
    def prepare_feature_vector(self, inputs): 
        country = self.profiles_country_fv.get_feature_vector(entry={'cc_num': inputs[0]['cc_num']})[0]
        is_transaction_abroad = transaction_abroad(pd.Series(data=inputs[0]['country']), pd.Series(data=country))
        
        feature_vector = self.fraud_fv.get_feature_vector(entry={'cc_num': inputs[0]['cc_num']},
                                               passed_features={
                                                  'amount': inputs[0]['amount'],
                                                  'category': inputs[0]['category'],
                                                  'is_transaction_abroad': is_transaction_abroad[0],
                                              },
                                              return_type="pandas")
        
        return feature_vector
    
    def monitor(self, inputs, feature_vector, prediction):
        pdf = feature_vector
        
        pdf['cc_num'] = inputs[0]['cc_num']
        pdf['event_time'] = pd.Timestamp.now()
        pdf['prediction'] = prediction
        
        self.monitoring_fg.multi_part_insert(pdf)


In [ ]:
# Get the dataset API for the current project
dataset_api = project.get_dataset_api()

# Specify the local file path of the Python script to be uploaded
local_script_path = "/tmp/predict_example.py"

# Upload the Python script to the "Models", and overwrite if it already exists
uploaded_file_path = dataset_api.upload(local_script_path, "Models", overwrite=True)

# Create the full path to the uploaded script for future reference
predictor_script_path = os.path.join("/Projects", project.name, uploaded_file_path)

In [ ]:
# Deploy the fraud model
deployment = model.deploy(
    name="fraudonlinedep1",
    script_file=predictor_script_path,
)

In [ ]:
deployment.start(await_running=300)

In [ ]:
# Print the name of the deployment
print("Deployment: " + deployment.name)

# Display information about the deployment
deployment.describe()

In [ ]:
# Test the model
deployment.predict(inputs = {
    'cc_num': 4738877982476642,
    'amount': 123,
    'category': 'Cash Withdrawal',
    'country': 'US'
})